In [1]:
import CHONK_cpp as ch
import numpy as np
import fastscapelib_fortran as fs
import lsdtopytools as lsd
import numba as nb
from matplotlib import pyplot as plt
%matplotlib inline
from IPython.display import clear_output, display
nrows = 100
ncols = 100
xres = 500
yres = 500
xmin = 0
xmax = ncols*yres
ymin = 0
ymax = nrows*xres
Z = np.random.rand(nrows,ncols)/100
Z = np.load("elev_breaking_stack.npy")
def check_stack(stack):
    checker = np.zeros(stack.shape, dtype = np.int32)
    incr = 0
    for i in stack:
        checker[i] = incr
        incr +=1
    return checker

In [5]:
fs.fastscape_init()
fs.fastscape_set_nx_ny(ncols,nrows)
fs.fastscape_setup()
fs.fastscape_set_xl_yl(xmax - xmin, ymax - ymin)
fs.fastscape_set_bc(1010)
fstctx = fs.fastscapecontext
fstctx.p = -1
fstctx.ibc = 1010



In [6]:
for i in range(1):
#     Z = np.random.rand(nrows,ncols)/100
    active = np.zeros((nrows,ncols), dtype = np.int)
    active[1:-1,:] = 1
    fstctx.h = Z.astype(np.float64).ravel()
    fs.find_stacks_with_intermediate_save ()
    stack = fstctx.stack0.astype('int') - 1
    rec = fstctx.rec0.astype('int') - 1
    prec = fstctx.rec.astype('int') - 1
    mrec = fstctx.mrec.astype('int').transpose() - 1
    mlengths = fstctx.mlrec.transpose()
    lengths = fstctx.length
    nodegraph = ch.NodeGraph(stack,rec, prec,lengths,mrec,mlengths, active.ravel().astype(int),Z.ravel(),xres,yres,nrows,ncols)

RuntimeError: receiver basin of rerouted pit is itself::pitnode:0 recnode:0

In [4]:
print(active.astype(np.bool))

[[False False False ... False False False]
 [ True  True  True ...  True  True  True]
 [ True  True  True ...  True  True  True]
 ...
 [ True  True  True ...  True  True  True]
 [ True  True  True ...  True  True  True]
 [False False False ... False False False]]


In [5]:
MF = nodegraph.get_MF_stack_full()

NameError: name 'nodegraph' is not defined

In [6]:
plt.imshow(check_stack(MF).reshape(nrows,ncols))

NameError: name 'MF' is not defined

In [9]:
lsd.raster_loader.save_raster(check_stack(MF).reshape(nrows,ncols),
   0,xres*ncols,yres*nrows,0,xres,"epsg:32635","nodegraphv2_stack.tif", fmt = 'GTIFF')
lsd.raster_loader.save_raster(Z.reshape(nrows,ncols),
   0,xres*ncols,yres*nrows,0,xres,"epsg:32635","nodegraphv2_topo.tif", fmt = 'GTIFF')
lsd.raster_loader.save_raster(np.arange(nrows*ncols).astype(np.int32).reshape(nrows,ncols),
   0,xres*ncols,yres*nrows,0,xres,"epsg:32635","nodegraphv2_ID.tif", fmt = 'GTIFF')